In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import ast
import pickle
import optuna
import warnings

Most of this code comes from Googles Training course on text classification:  https://developers.google.com/machine-learning/guides/text-classification.  This has been adapted and improved to include k-folds cross validation to produce a more consistent result in training given that the volume of training data is low.

### Step 1 - Import Train and Test Data and convert to required data type

In [3]:
# Load training set from CSV, convert CSV text to list and join into a single string
train_data = pd.read_csv("train_data.csv")
train_data['preprocessed_text'] = train_data['preprocessed_text'].apply(ast.literal_eval)
train_data['preprocessed_text'] = train_data['preprocessed_text'].apply(' '.join)
train_texts = train_data['preprocessed_text'].to_numpy()

# Load test set from CSV, convert CSV text to list and join into a single string
test_data = pd.read_csv("test_data.csv")
test_data['preprocessed_text'] = test_data['preprocessed_text'].apply(ast.literal_eval)
test_data['preprocessed_text'] = test_data['preprocessed_text'].apply(' '.join)
test_texts = test_data['preprocessed_text'].to_numpy()


In [14]:
# Create dictionary of integer values for the labels so we can convert back at a later dat
def create_label_dict(df, col_name):
    unique_values = df[col_name].unique()
    value_dict = {value: i for i, value in enumerate(unique_values)}
    return value_dict

label_dict = create_label_dict(train_data, 'clause_type')

# convert existing text labels to integers 1-10 using dictionary
def transform_column(df, col_name, mapping_dict):
    return df[col_name].map(mapping_dict).values.astype(np.int64)

train_labels = transform_column(train_data, 'clause_type', label_dict)
test_labels = transform_column(test_data, 'clause_type', label_dict)


### Step 2 - Functions for Vectorisation

In [5]:
# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000
# Whether text should be split into word or character n-grams. One of 'word', 'char'.
TOKEN_MODE = 'word'
# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(x_train_fold, y_train_fold, x_val_fold,**kwargs):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        x_train_fold: list, training text strings.
        y_train_fold: np.ndarray, training labels (integers).
        x_val_fold: list, validation text strings.

    # Returns
        x_train: vectorized training texts
        x_val: vectorized validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
        'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
        'dtype': 'int32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,  # Split text into word tokens.
        'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train_fold = vectorizer.fit_transform(x_train_fold).astype(np.float32)  # Fixed variable name

    # Vectorize validation texts.
    x_val_fold = vectorizer.transform(x_val_fold).astype(np.float32)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train_fold.shape[1]))
    selector.fit(x_train_fold, y_train_fold)
    x_train = selector.transform(x_train_fold).astype('float32')
    x_val = selector.transform(x_val_fold).astype('float32')
    
    # Save vectorizer and selector to disk for future use
    with open('vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)

    with open('selector.pkl', 'wb') as f:
        pickle.dump(selector, f)
    
    # Return both the vectorized texts
    return x_train, x_val

In [6]:
def vectorize_production_data(prod_texts,**kwargs):
    """Vectorizes new production texts using the pre-trained vectorizer and selector.

    # Arguments
        prod_texts: list, new production text strings.

    # Returns
        x_prod: vectorized production texts.
    """

    # Load the saved vectorizer and feature selector
    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    with open('selector.pkl', 'rb') as f:
        selector = pickle.load(f)

    # Vectorize production texts using the trained vectorizer
    x_prod = vectorizer.transform(prod_texts).astype(np.float32)

    # Apply feature selection using the trained selector
    x_prod = selector.transform(x_prod).astype(np.float32)

    return x_prod

### Step 3 - Define Function to create last layer according to data parameters

In [7]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.
    # Arguments
        num_classes: int, number of classes.
    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

### Step 4 - Define function to build simple multi-layer perceptron  

In [8]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    # Define the input shape using Input layer
    model.add(Input(shape=input_shape))

    # Add the first Dense layer (input layer) without Dropout yet
    model.add(Dense(units=units, activation='relu'))

    # Add Dropout and subsequent layers
    for _ in range(layers - 1):
        model.add(Dropout(rate=dropout_rate))  # Add Dropout after Dense layers
        model.add(Dense(units=units, activation='relu'))

    # Output layer
    model.add(Dense(units=op_units, activation=op_activation))

    return model

### Step 5 - Create a function to train the n_gram model

In [9]:
# train model with kfolds validation
def train_ngram_model_kfolds(data,k=5,
                            learning_rate=1e-3,
                            epochs=1000,
                            batch_size=128,
                            layers=2,
                            units=64,
                            dropout_rate=0.2,
                            num_classes=10,
                            **kwargs):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        """Trains n-gram model on the given dataset.
    
        # Arguments
            k: no of folds 
            data: tuples of training and test texts and labels.
            learning_rate: float, learning rate for training model.
            epochs: int, number of epochs.
            batch_size: int, number of samples per batch.
            layers: int, number of `Dense` layers in the model.
            units: int, output dimension of Dense layers in the model.
            dropout_rate: float: percentage of input to drop at Dropout layers.
            num_classes: int, number of output classes.
        """
        # Get the data.
        train_texts, train_labels = data
        
        # Prepare k-fold cross-validation
        kfold = KFold(n_splits=k, shuffle=True, random_state=42)
        
        # Store validation results for each fold
        val_accuracies = []
        val_losses = []
    
        # Perform K-Fold Cross-Validation
        for train_idx, val_idx in kfold.split(train_texts):
            # Split data into training and validation for this fold
            # K-fold cross-validation
            x_train_fold = train_texts[train_idx]
            y_train_fold = train_labels[train_idx]
            x_val_fold = train_texts[val_idx]
            y_val_fold = train_labels[val_idx]
            
           # Vectorize texts.
            x_train, x_val = ngram_vectorize(x_train_fold, y_train_fold, x_val_fold,**kwargs)             
            
            # Check for empty tensors
            if x_train.shape[0] == 0:
                print("Training data tensor is empty")
                print("kwargs:")
                for key, value in kwargs.items():
                    print(f"{key}: {value}")
            if x_val.shape[0] == 0:
                print("Validation data tensor is empty")
                print("kwargs:")
                for key, value in kwargs.items():
                    print(f"{key}: {value}")
            
            # Create model instance.
            model = mlp_model(layers=layers,
                              units=units,
                              dropout_rate=dropout_rate,
                              input_shape=x_train.shape[1:],
                              num_classes=num_classes)
            
            loss = 'sparse_categorical_crossentropy'
            optimizer = Adam(learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        
            # Create callback for early stopping on validation loss. If the loss does not decrease in two consecutive tries, stop training.
            callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
            
            # Train and validate the model on this fold
            history = model.fit(
                x_train,
                y_train_fold,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=(x_val, y_val_fold),
                verbose=0,
                batch_size=batch_size
            )
            # Collect the last validation accuracy and loss
            history = history.history
            val_accuracies.append(history['val_accuracy'][-1])
            val_losses.append(history['val_loss'][-1])
        
        # Compute average validation accuracy and loss
        avg_val_accuracy = np.mean(val_accuracies)
        avg_val_loss = np.mean(val_losses)
        
        # Print results.
        print('Average Validation Accuracy: {:.4f}, Average Loss: {:.4f}'.format(avg_val_accuracy, avg_val_loss))
    
        # Save model.
        model.save('Clause_class_model.keras')
        
    return avg_val_accuracy, avg_val_loss

### Step 6 - Build Model and train model

In [10]:
data = (train_texts, train_labels)
train_ngram_model_kfolds(data)

Average Validation Accuracy: 0.8679, Average Loss: 0.3902


(0.8678576231002808, 0.3901568830013275)

### Step 7 - Make predictions on test data prior to Hyperparameter Optimisation

In [11]:
# Vectorize the production data using the previously defined function
x_prod = vectorize_production_data(test_texts)

# Make predictions on the production data using the trained model
model = load_model('Clause_class_model.keras')
predictions = model.predict(x_prod)

# Step 3: Convert predicted probabilities to class labels. N.B. Its argmax because you get a softmax output so you want the most probab;y class
predicted_classes = np.argmax(predictions, axis=1)

# Step 4: Calculate accuracy by comparing predicted classes with true labels
accuracy = accuracy_score(test_labels, predicted_classes)

# Step 5: Output the accuracy
print(f"Accuracy on test data: {accuracy:.4f}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Accuracy on test data: 0.8903


### Step 8 - Tune Hyperparameters

In [16]:
def objective(trial):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        data = (train_texts, train_labels)
        # Define the hyperparameters to optimize
        learning_rate = trial.suggest_float("learning_rate", 1e-5,1e-1)
        layers = trial.suggest_int("layers", 2, 6)
        units = trial.suggest_categorical("units", [64,128])
        dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
        NGRAM_RANGE = trial.suggest_categorical('N_GRAM_RANGE', [(1, 1), (1, 2), (2, 2),(1, 3),(2, 3),(3, 3),(1, 4),(2, 4),(3, 4),(4, 4)])  
        TOP_K = trial.suggest_int("TOP_K",10000,40000)
        MIN_DOCUMENT_FREQUENCY = trial.suggest_int("MIN_DOCUMENT_FREQUENCY", 1, 2)
            
        # Call the train_ngram_model_kfolds function with the sampled hyperparameters
        avg_val_accuracy, avg_val_loss = train_ngram_model_kfolds(data, learning_rate=learning_rate,layers=layers,units=units,dropout_rate=dropout_rate,num_classes=10, NGRAM_RANGE=NGRAM_RANGE,TOP_K=TOP_K, MIN_DOCUMENT_FREQUENCY=MIN_DOCUMENT_FREQUENCY)

    # Since Optuna minimizes the objective, we may want to minimize -accuracy if maximizing accuracy
    return avg_val_accuracy  # Maximize accuracy by minimizing negative accuracy

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print the best result
print("Best value:", study.best_value)
print("Best parameters:", study.best_params)

[I 2024-09-22 20:38:28,394] A new study created in memory with name: no-name-3e58f3d2-7181-4332-ad21-cfd0290b9e9b
[I 2024-09-22 20:38:34,623] Trial 0 finished with value: 0.862898302078247 and parameters: {'learning_rate': 0.01819183800107648, 'layers': 2, 'units': 128, 'dropout_rate': 0.44704539127666726, 'N_GRAM_RANGE': (2, 4), 'TOP_K': 29311, 'MIN_DOCUMENT_FREQUENCY': 1}. Best is trial 0 with value: 0.862898302078247.


Average Validation Accuracy: 0.8629, Average Loss: 0.5415


[I 2024-09-22 20:38:43,020] Trial 1 finished with value: 0.6916774034500122 and parameters: {'learning_rate': 0.06955646357038357, 'layers': 4, 'units': 64, 'dropout_rate': 0.36596654979199195, 'N_GRAM_RANGE': (4, 4), 'TOP_K': 26933, 'MIN_DOCUMENT_FREQUENCY': 2}. Best is trial 0 with value: 0.862898302078247.


Average Validation Accuracy: 0.6917, Average Loss: 0.7249


[I 2024-09-22 20:38:51,946] Trial 2 finished with value: 0.10919947028160096 and parameters: {'learning_rate': 0.06038854030200841, 'layers': 6, 'units': 128, 'dropout_rate': 0.4806182562657614, 'N_GRAM_RANGE': (3, 3), 'TOP_K': 15840, 'MIN_DOCUMENT_FREQUENCY': 1}. Best is trial 0 with value: 0.862898302078247.


...

Average Validation Accuracy: 0.8679, Average Loss: 0.4160


[I 2024-09-22 21:03:06,156] Trial 98 finished with value: 0.8678691744804382 and parameters: {'learning_rate': 0.004946611557028899, 'layers': 2, 'units': 128, 'dropout_rate': 0.4595812196676481, 'N_GRAM_RANGE': (1, 1), 'TOP_K': 36774, 'MIN_DOCUMENT_FREQUENCY': 2}. Best is trial 71 with value: 0.8765551924705506.


Average Validation Accuracy: 0.8679, Average Loss: 0.4273


[I 2024-09-22 21:03:13,225] Trial 99 finished with value: 0.867246127128601 and parameters: {'learning_rate': 0.011251613875446287, 'layers': 3, 'units': 128, 'dropout_rate': 0.4864732405699365, 'N_GRAM_RANGE': (2, 4), 'TOP_K': 30870, 'MIN_DOCUMENT_FREQUENCY': 2}. Best is trial 71 with value: 0.8765551924705506.


Average Validation Accuracy: 0.8672, Average Loss: 0.4850
Best value: 0.8765551924705506
Best parameters: {'learning_rate': 0.0018962810428611815, 'layers': 2, 'units': 128, 'dropout_rate': 0.47693769062401753, 'N_GRAM_RANGE': (2, 4), 'TOP_K': 34293, 'MIN_DOCUMENT_FREQUENCY': 2}


### Step 10 - Test Model


In [17]:
# Train vectoriser and model
data = (train_texts, train_labels)
train_ngram_model_kfolds(data,learning_rate = 0.0018962810428611815, layers = 2, units = 128, dropout_rate = 0.47693769062401753, N_GRAM_RANGE = (2, 4), TOP_K= 34293, MIN_DOCUMENT_FREQUENCY= 2)

# Vectorize the production data using the previously defined function
x_prod = vectorize_production_data(test_texts)

# Make predictions on the production data using the trained model
model = load_model('Clause_class_model.keras')
predictions = model.predict(x_prod)

# Step 3: Convert predicted probabilities to class labels. N.B. Its argmax because you get a softmax output so you want the most highly probable class
predicted_classes = np.argmax(predictions, axis=1)

# Step 4: Calculate accuracy by comparing predicted classes with true labels
accuracy = accuracy_score(test_labels, predicted_classes)

# Step 5: Output the accuracy
print(f"Accuracy on test data: {accuracy:.4f}")


Average Validation Accuracy: 0.8685, Average Loss: 0.3810
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Accuracy on test data: 0.8792
